In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import time
import pandas as pd
import sys
import types
from sklearn.preprocessing import StandardScaler
import math
import keras
import tensorflow as tf
from matplotlib import pyplot as plt
from scipy import misc
from sklearn import preprocessing
import datetime
import sklearn
tf.keras.backend.set_floatx('float64')
df = pd.read_excel('/storage/data.xlsx').drop('Date',axis=1)[:3770].fillna(method='ffill').astype('float64')

def test_step(modell):
    for it,data in enumerate(iter(modell.test_data),start=0):
        model_inp,test_out = data
        if it is 0:
            model_out_conc = tf.reshape(modell.modell(model_inp,training = False),shape=(1,3))
            test_out_conc = tf.reshape(test_out,shape=(1,3))
        else:

            model_out_conc = tf.concat((model_out_conc,tf.reshape(modell.modell(model_inp,training=False),shape=(1,3))), axis=0)
            test_out = tf.reshape(test_out,shape=(1,3))
            test_out_conc = tf.concat((test_out_conc,test_out),axis=0)
    return test_out_conc,model_out_conc

class helpful:
    def Preprocessin(self,windowlength = 22, prediction_length = 3, max_wlength=22, outcol = 3):
        df = self.df
        dftrain=df.copy()[257:2675]
        dfvalid=df.copy()[2675:3500]
        dftest = df.copy()[3500:]
        trainarray=dftrain.iloc[:,outcol].copy()                                   #extract array of particular feature  
        validarray=dfvalid.iloc[:,outcol].copy()                                    
        testarray=dftest.iloc[:,outcol].copy()              #,tmean,tvar=tanz(arrayzz[:,3].copy(), True)   
        transformerr = sklearn.preprocessing.StandardScaler()
        self.transformerr = transformerr.fit(np.array(trainarray).reshape(-1,1))
        trainarray = self.transformerr.transform(np.array(trainarray).reshape(-1,1))
        validarray = self.transformerr.transform(np.array(validarray).reshape(-1,1))
        testarray = self.transformerr.transform(np.array(testarray).reshape(-1,1))


        pagestrain = trainarray.shape[0] - max_wlength - prediction_length
        pagesvalid = validarray.shape[0]  - max_wlength - prediction_length
        pagestest = testarray.shape[0] - max_wlength - prediction_length-2

        OUTPUT_train = np.asarray([[np.array(trainarray)[i+k+max_wlength] for i in range(prediction_length)] for k in range( pagestrain)])
        OUTPUT_val = np.asarray([[np.array(validarray)[i+k+max_wlength] for i in range(prediction_length)] for k in range(pagesvalid)])
        OUTPUT_test = np.asarray([[ np.array(testarray)[i+k+max_wlength] for i in range(prediction_length )] for k in range(pagestest)])
        for feature in range(np.array(dftrain).shape[1]):
            sys.stdout.write("\r{}" .format(feature/dftrain.shape[1]))
            sys.stdout.flush()

            trainarray=np.asarray(dftrain.iloc[:,feature].copy())                                   #extract array of particular feature  
            validarray=np.asarray(dfvalid.iloc[:,feature].copy())                                    
            testarray=np.asarray(dftest.iloc[:,feature].copy())  
            transformer = sklearn.preprocessing.StandardScaler()
            transformer = transformer.fit(np.array(trainarray).reshape(-1,1))
            trainarray = transformer.transform(np.array(trainarray).reshape(-1,1))
            validarray = transformer.transform(np.array(validarray).reshape(-1,1))
            testarray = transformer.transform(np.array(testarray).reshape(-1,1))
            del transformer


            #trainarray, mean, std = tanz(np.array(trainarray))
            #validarray = tanzws(np.array(validarray),mean,std)
            #testarray = tanzws(np.array(testarray),mean,std)
            #save 2d time series data as  3d list with batch_size=windowlength
            trainarray = np.array([[[ np.array(trainarray)[i+k+max_wlength-windowlength] for i in  range(windowlength)] for t in range(1)] for k in range(pagestrain)])
            validarray = np.array([[[np.array(validarray)[i+k+max_wlength-windowlength] for i in range(windowlength)] for t in range(1)] for k in range(pagesvalid)]) 
            testarray = np.array([[[ np.array(testarray)[i+k+max_wlength-windowlength] for i in range(windowlength)]for t in range(1)] for k in range(pagestest)])
            if feature == 0:
                INPUT_train_lstm = trainarray.copy()
                INPUT_valid_lstm = validarray.copy()
                INPUT_test_lstm = testarray.copy()
            else:
                INPUT_train_lstm = np.append(INPUT_train_lstm,trainarray,axis=1)
                INPUT_valid_lstm = np.append(INPUT_valid_lstm,validarray,axis=1)
                INPUT_test_lstm=np.append(INPUT_test_lstm,testarray,axis=1)

        INPUT_train_lstm=INPUT_train_lstm.swapaxes(1,2)
        INPUT_valid_lstm=INPUT_valid_lstm.swapaxes(1,2)
        INPUT_test_lstm=INPUT_test_lstm.swapaxes(1,2)
        INPUT_test_lstm[0,0]
        return  np.squeeze(INPUT_train_lstm),np.squeeze(INPUT_test_lstm),np.squeeze(INPUT_valid_lstm),np.squeeze(OUTPUT_val), np.squeeze(np.array(OUTPUT_train)), np.squeeze(np.array(OUTPUT_test))


    def parallel_window(self,batchsize = 48, windowlength=[22,14], prediction_length=3,max_wlength=22,outcol=3):
        inptrain22, inptest22 , inpvalid22, outvalid, outtrain, outtest = self.Preprocessin(windowlength=windowlength[0]) 
        inptrain22.shape
        inptrain14, inptest14 , inpvalid14, outvalid14, outtrain14, outtest14 = self.Preprocessin(windowlength=windowlength[1]) 
        inptrain = np.concatenate([inptrain22,inptrain14],axis=1).astype('float64')
        inptest = np.concatenate([inptest22,inptest14],axis=1).astype('float64')
        inpvalid = np.concatenate([inpvalid22,inpvalid14],axis=1).astype('float64')

        train_data = tf.data.Dataset.from_tensor_slices((inptrain,outtrain))
        self.train_data = train_data.cache().batch(batchsize).repeat(1)
        valid_data = tf.data.Dataset.from_tensor_slices((inpvalid,outvalid))
        self.valid_data = valid_data.cache().batch(batchsize).repeat(1)    
        test_data = tf.data.Dataset.from_tensor_slices((inptest,outtest))
        self.test_data = test_data.cache().batch(1).repeat(1)
        
    def windowbatch(self, windowlength=22, prediction_length=3,max_wlength=22,outcol=3):
        inptrain, inptest , inpvalid, outvalid, outtrain, outtest = self.Preprocessin() 


        train_data = tf.cast(tf.data.Dataset.from_tensor_slices((inptrain,outtrain)),tf.float64)
        self.train_data = train_data.cache().batch(64).repeat(1)
        valid_data = tf.cast(tf.data.Dataset.from_tensor_slices((inpvalid,outvalid)),tf.float64)
        self.valid_data = valid_data.cache().batch(64).repeat(1)    
        test_data = tf.cast(tf.data.Dataset.from_tensor_slices((inptest,outtest)),tf.float64)
        self.test_data = test_data.cache().batch(1).repeat(1)

        print('\r all data batched and ready \n')

"""        
        #trainarray, self.train_mean, self.train_std = tanz(np.array(trainarray))
        #validarray = tanzws(np.array(validarray),self.train_mean,self.train_std)
        #testarray = tanzws(np.array(testarray),self.train_mean,self.train_std)
        transformer = sklearn.preprocessing.StandardScaler()
        self.transformer = transformer.fit(np.array(trainarray).reshape(-1,1))
        trainarray = self.transformer.transform(np.array(trainarray).reshape(-1,1))
        validarray = self.transformer.transform(np.array(validarray).reshape(-1,1))
        testarray = self.transformer.transform(np.array(testarray).reshape(-1,1))
"""

class MODELL(helpful):
    def __init__(self):
        self.batchsize = 16
        self.train_mean = None
        self.train_std = None
        self.train_data = None
        self.valid_data = None
        self.test_data = None
        self.epochz = None
        self.df = df
        self.optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.000028, decay=0.7,momentum = 0.99)
        self.initz = tf.keras.initializers.glorot_uniform(seed=None)
        self.loss_mse = tf.keras.losses.MeanSquaredError()
        self.transformerr = None
        self.loss_metric = tf.keras.metrics.Mean(name='loss_metric')
        self.loss_metric_valid = tf.keras.metrics.Mean(name='loss_metric_valid')
        
        
        
    def model_parallel(self):
        inp = tf.keras.layers.Input(shape=(36,df.shape[1]))
        x1,x2 = tf.split(inp,[22,14],axis=1)
        x1 = tf.keras.layers.Conv1D(480,kernel_initializer=self.initz,kernel_size=5)(x1)
        x1
        x1 = tf.keras.layers.LeakyReLU(alpha=0.4)(x1)
        x1
        x1 = tf.keras.layers.Conv1D(264,kernel_initializer=self.initz,kernel_size=4)(x1)
        
        x1 = tf.keras.layers.LeakyReLU(alpha=0.25)(x1)
        
        x1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(x1)
        
        x1 = tf.keras.layers.Conv1D(148,kernel_initializer=self.initz,kernel_size=3)(x1)
        
        x1 = tf.keras.layers.LeakyReLU(alpha=0.15)(x1)
        
        x1 = tf.keras.layers.LSTM(128,kernel_initializer=self.initz, return_sequences = True)(x1)
        
        x1 = tf.keras.layers.LeakyReLU(alpha=0.25)(x1)

        x2 = tf.keras.layers.Conv1D(256,kernel_initializer=self.initz,kernel_size=2)(x2)
        x2 = tf.keras.layers.LeakyReLU(alpha=0.4)(x2)
        x2 = tf.keras.layers.Conv1D(128,kernel_initializer=self.initz,kernel_size=2)(x2)
        x2 = tf.keras.layers.LeakyReLU(alpha=0.2)(x2)
        x2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None)(x2)
        x2 = tf.keras.layers.Conv1D(96,kernel_initializer=self.initz,kernel_size=2)(x2)
        x2 = tf.keras.layers.LeakyReLU(alpha=0.1)(x2)
        x2 = tf.keras.layers.LSTM(256,kernel_initializer=self.initz, return_sequences = True)(x2)
        x2
        x2 = tf.keras.layers.LeakyReLU(alpha=0.15)(x2)
        x1 = tf.keras.layers.Flatten()(x1)
        x2 = tf.keras.layers.Flatten()(x2)

        x = tf.concat([x1,x2],axis=1)
        x = tf.keras.layers.Dense(128,use_bias = False)(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.15)(x)
        x = tf.keras.layers.Dropout(0.45)(x)

        x = tf.keras.layers.Dense(64,use_bias = False)(x)
        x = tf.keras.layers.LeakyReLU(alpha=0.15)(x)
        x = tf.keras.layers.Dropout(0.35)(x)

        out = tf.keras.layers.Dense(3,use_bias = False)(x)
        self.modell=tf.keras.Model(inp,out)
        self.modell_weightz = self.modell.trainable_variables

    def inverse_transform(self):
        return self.inversetanz(self.model_test_pred,self.train_mean,self.train_std), self.inversetanz(self.model_test_out,self.train_mean,self.train_std)

    def reset_weights_and_model(self):
        self.modell.trainable_weights = self.modell_weightz
    

    #@tf.function
    def train_step(self,data):
        inp,real_out = data
        with tf.GradientTape() as lstm_tape:
            model_out = self.modell(inp,training=True)
            loss = tf.cast(self.loss_mse(model_out, real_out),tf.float64)

        gradients = lstm_tape.gradient(loss, self.modell.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.modell.trainable_variables))
        return self.loss_metric(loss)
    
    #@tf.function
    def valid_step(self,data_v):
        inp_train_v,real_out_v = data_v
        model_out_v = self.modell(inp_train_v,training=False)
        loss_value_v = self.loss_mse(model_out_v, real_out_v)
        return self.loss_metric_valid(loss_value_v)

    
    def test_step(self):
        for it,data in enumerate(iter(self.test_data),start=0):
            model_inp,test_out = data
            if it is 0:
                model_out_conc = tf.reshape(self.modell(model_inp,training = False),shape=(1,3))
                test_out_conc = tf.reshape(test_out,shape=(1,3))
            else:
                
                model_out_conc = tf.concat((model_out_conc,tf.reshape(self.modell(model_inp,training=False),shape=(1,3))), axis=0)
                test_out = tf.reshape(test_out,shape=(1,3))
                test_out_conc = tf.concat((test_out_conc,test_out),axis=0)
        self.model_test_pred = model_out_conc
        self.model_test_out = test_out_conc


    def model_test_out(self,lose_bias=False):
        self.test_step()
        testout = self.transformerr.inverse_transform(self.model_test_out)
        testpred = self.transformerr.inverse_transform(self.model_test_pred)
        real = list()
        predz = list()

        for i in range(testout.shape[0]):
            if i%3 is 0:
                real.extend(testout[i])
                predz.extend(testpred[i])
        predz = np.array(predz)
        real = np.array(real)
        if lose_bias:
            predz = predz - (predz.mean() - real.mean() )
        plt.plot(real,label='real')
        plt.plot(predz,label='pred')
        plt.legend()

    
    
    def trainingz(self):
        current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
        test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
        valid_log_dir = 'logs/gradient_tape/' + current_time + '/valid'
        train_summary_writer = tf.summary.create_file_writer(train_log_dir)
        test_summary_writer = tf.summary.create_file_writer(test_log_dir)
        valid_summary_writer = tf.summary.create_file_writer(valid_log_dir)

        for epoch in range(self.epochz):
            start = time.time()

            for data in self.train_data:
                loss_ = self.train_step(data)              
            with train_summary_writer.as_default():
                tf.summary.scalar('loss_metric', self.loss_metric.result(), step=epoch)


            for data_v in self.valid_data:
                self.valid_step(data_v)
            with valid_summary_writer.as_default():
                tf.summary.scalar('loss_metric_valid', self.loss_metric_valid.result(), step=epoch)

            template = 'Sec : {} \n Epoch {} ---- Loss: {}  ----  Val_Loss: {}'
            tf.print('Epoch', epoch, ': Time', time.time()-start, ': loss', self.loss_metric.result(), '; valid_loss', self.loss_metric_valid.result())
            self.loss_metric.reset_states()
            self.loss_metric_valid.reset_states()
        return epoch, self.loss_metric.result() , self.loss_metric_valid.result()
modell = MODELL()
modell.model_parallel()
modell.epochz = 3
modell.df = df
modell.parallel_window()
modell.trainingz()

Using TensorFlow backend.


0.98360655737704924Epoch 0 : Time 10.990955114364624 : loss 0.97286774858832359 ; valid_loss 4.6291896139874176
Epoch 1 : Time 10.754860401153564 : loss 1.247023033797741 ; valid_loss 2.1545129244161
Epoch 2 : Time 10.774378061294556 : loss 1.1005824652314187 ; valid_loss 0.69764198581962023
Epoch 3 : Time 10.756994724273682 : loss 0.62783252716064453 ; valid_loss 1.1249279237407095
Epoch 4 : Time 10.724502325057983 : loss 0.484947469830513 ; valid_loss 1.7557484380462591
Epoch 5 : Time 10.737718343734741 : loss 0.45159276679158211 ; valid_loss 1.3277852312278222
Epoch 6 : Time 10.764772891998291 : loss 0.390278914347291 ; valid_loss 0.893753783284303
Epoch 7 : Time 10.617450952529907 : loss 0.34778528116643431 ; valid_loss 0.64264728764400758
Epoch 8 : Time 10.65851616859436 : loss 0.2894246181845665 ; valid_loss 0.59958607155610533
Epoch 9 : Time 10.746488332748413 : loss 0.255523500777781 ; valid_loss 0.67511584281044845
Epoch 10 : Time 10.683893918991089 : loss 0.2516524036601186 ;

KeyboardInterrupt: 